In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import json
import pandas as pd
from pathlib import Path
from copy import deepcopy
from functools import partial

from bellek.qa.jerxrm import answer_question, answer_question_with_reasoning
from bellek.utils import set_seed, jprint
from bellek.musique.singlehop import benchmark

set_seed(89)

/home/pc/anaconda3/envs/baris/lib/python3.10/site-packages/torch/cuda/__init__.py:619: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [3]:
from tqdm.auto import tqdm
tqdm.pandas()

In [4]:
from bellek.musique.constants import ABLATION_RECORD_IDS

df = pd.read_json('../../data/generated/musique-evaluation/dataset.jsonl', orient='records', lines=True)
df = df.set_index('id', drop=False).loc[ABLATION_RECORD_IDS].copy().reset_index(drop=True)

# df = df.sample(10)

print(len(df))
df.head()

100


,id,paragraphs,question,question_decomposition,answer,answer_aliases,answerable,answers
0,2hop__575188_342798,"[{'idx': 0, 'title': 'Liliana Mumy', 'paragrap...",Who is the child of Mahmoud Mirza's father?,"[{'id': 575188, 'question': 'Mahmoud Mirza >> ...",Ahmad Shah Qajar,[Ahmad Shah Qajar],True,[Ahmad Shah Qajar]
1,2hop__731584_700117,"[{'idx': 0, 'title': 'KAPE', 'paragraph_text':...",In which county is the city to which KKVU is l...,"[{'id': 731584, 'question': 'KKVU >> licensed ...",Berrien County,[Berrien County],True,[Berrien County]
2,2hop__690412_526810,"[{'idx': 0, 'title': 'Cabramatta Creek', 'para...",For what river does the river on which Pa Sak ...,"[{'id': 690412, 'question': 'Pa Sak Jolasid Da...",Chao Phraya River,[Chao Phraya River],True,[Chao Phraya River]
3,2hop__263638_69048,"[{'idx': 0, 'title': 'Michael J. Barron', 'par...",Who is the Chief Judge of the Tebesa Nemine's ...,"[{'id': 263638, 'question': 'Tebesa Nemine >> ...",Honorable Justice Abiodun Smith,[Honorable Justice Abiodun Smith],True,[Honorable Justice Abiodun Smith]
4,2hop__142842_68489,"[{'idx': 0, 'title': 'Perfect Night: Live in L...",Who did the performer of Night Rocker play on ...,"[{'id': 142842, 'question': 'Which performer r...",Snapper Foster,[Snapper Foster],True,[Snapper Foster]


In [5]:
# qd_df = pd.read_json('../../data/generated/musique-evaluation/question-decomposition.jsonl', orient='records', lines=True)
# df = pd.merge(df.drop(columns=['question', 'question_decomposition']), qd_df, on='id', suffixes=('', ''))
# print(df.shape)
# df.head()

In [6]:
df['paragraphs'] = df['paragraphs'].map(lambda ps: [p for p in ps if p['is_supporting']])

In [7]:
perfect_retrieval_func = lambda docs, query: [doc for doc in docs if doc['is_supporting']]

In [8]:
completion_params = {
    "temperature": 0.1,
}

In [9]:
import magentic

def qa_func(
    context: str,
    question: str,
): 
    with magentic.OpenaiChatModel("gpt-4o-mini", **completion_params):
        return answer_question(context, question)

In [10]:
N_RUNS = 1
results = []

## llama-zero-shot

In [11]:
from bellek.jerx.fewshot.llm import make_kg_triplet_extract_fn, DEFAULT_JERX_SYSTEM_MESSAGE_FOR_LLAMA

prefix_messages = [
    dict(role="system", content=DEFAULT_JERX_SYSTEM_MESSAGE_FOR_LLAMA),
]
extract_kg_triplets = make_kg_triplet_extract_fn(model='llama3-8b-togetherai', prefix_messages=prefix_messages, completion_params=completion_params)

def replace_paragraphs(row):
    new_paragraphs = []
    for p in row['paragraphs']:
        p = deepcopy(p) 
        triplets_str = '\n'.join(" | ".join(triplet) for triplet in extract_kg_triplets(p['paragraph_text']))
        p['paragraph_text'] = '\n'.join(["# Entity-relation-entity triplets", triplets_str])
        new_paragraphs.append(p)
    row['paragraphs'] = new_paragraphs
    return row

df_llama_zs = df.progress_apply(replace_paragraphs, axis=1) 
df_llama_zs.head()
print(df_llama_zs.iloc[0]['paragraphs'][0]['paragraph_text'])

  0%|          | 0/100 [00:00<?, ?it/s]

# Entity-relation-entity triplets
Amanollah Khan Zia' os-Soltan | was | Iranian aristocrat
Amanollah Khan Zia' os-Soltan | was | politician
Amanollah Khan Zia' os-Soltan | served | Qajar court


In [12]:
for i in range(1,N_RUNS+1):
    df_llama_zs, scores = benchmark(df_llama_zs, qa_func, perfect_retrieval_func, ignore_errors=True)
    results.append({**scores, "retrieval": "groundtruth", "context": "triplets", "jerx": "llama-zero-shot", "run": i})
    jprint(scores)

  0%|          | 0/100 [00:00<?, ?it/s]

{
  "exact_match": 0.56,
  "f1": 0.6286153846153845,
  "fuzzy_match": 0.66
}


## llama-few-shot

In [13]:
def replace_paragraphs_with_offline_triplets(df, jerx_file):
    df = df.copy()

    jerx_df = pd.read_json(jerx_file, lines=True)
    jerx_mapping = {(row['id'], row['paragraph_idx']): row['generation'] for _, row in jerx_df.iterrows()}

    def replace_paragraphs(row):
        new_paragraphs = []
        for p in row['paragraphs']:
            p = deepcopy(p) 
            triplets_str = jerx_mapping[(row['id'], p['idx'])].strip()
            p['paragraph_text'] = triplets_str
            new_paragraphs.append(p)
        row['paragraphs'] = new_paragraphs
        return row

    return df.progress_apply(replace_paragraphs, axis=1) 

In [14]:
from bellek.jerx.fewshot.llm import make_kg_triplet_extract_fn

extract_kg_triplets = make_kg_triplet_extract_fn(model='llama3-8b-togetherai', completion_params=completion_params)

def replace_paragraphs(row):
    new_paragraphs = []
    for p in row['paragraphs']:
        p = deepcopy(p) 
        triplets_str = '\n'.join(" | ".join(triplet) for triplet in extract_kg_triplets(p['paragraph_text']))
        p['paragraph_text'] = '\n'.join(["# Entity-relation-entity triplets", triplets_str])
        new_paragraphs.append(p)
    row['paragraphs'] = new_paragraphs
    return row

df_llama_fs = df.progress_apply(replace_paragraphs, axis=1) 
df_llama_fs.head()
print(df_llama_fs.iloc[0]['paragraphs'][0]['paragraph_text'])

  0%|          | 0/100 [00:00<?, ?it/s]

# Entity-relation-entity triplets
Amanollah Khan Zia' os-Soltan | occupation | Iranian aristocrat
Amanollah Khan Zia' os-Soltan | occupation | politician
Amanollah Khan Zia' os-Soltan | affiliation | Qajar court


In [15]:
# df_llama_fs = replace_paragraphs_with_offline_triplets(df, Path("../../data/raw/musique-evaluation/jerx-inferences/llama3-base.jsonl"))
# df_llama_fs.head()

In [16]:
for i in range(1,N_RUNS+1):
    df_llama_fs, scores = benchmark(df_llama_fs, qa_func, perfect_retrieval_func, ignore_errors=True)
    results.append({**scores, "retrieval": "groundtruth", "context": "triplets", "jerx": "llama-few-shot", "run": i})
    jprint(scores)

  0%|          | 0/100 [00:00<?, ?it/s]

{
  "exact_match": 0.54,
  "f1": 0.6564862914862915,
  "fuzzy_match": 0.7
}


In [17]:
# _, scores = benchmark(df_llama_base, answer_question_with_reasoning, perfect_retrieval_func, ignore_errors=True)
# jprint(scores)

## llama-sft

In [18]:
df_llama_sft = replace_paragraphs_with_offline_triplets(df, Path("../../data/raw/musique-evaluation/jerx-inferences/llama3-sft-aw7ihmbc-ablation.jsonl"))
df_llama_sft.head()

  0%|          | 0/100 [00:00<?, ?it/s]

,id,paragraphs,question,question_decomposition,answer,answer_aliases,answerable,answers
0,2hop__575188_342798,"[{'idx': 7, 'title': 'Amanollah Khan Zia' os-S...",Who is the child of Mahmoud Mirza's father?,"[{'id': 575188, 'question': 'Mahmoud Mirza >> ...",Ahmad Shah Qajar,[Ahmad Shah Qajar],True,[Ahmad Shah Qajar]
1,2hop__731584_700117,"[{'idx': 1, 'title': 'KKVU', 'paragraph_text':...",In which county is the city to which KKVU is l...,"[{'id': 731584, 'question': 'KKVU >> licensed ...",Berrien County,[Berrien County],True,[Berrien County]
2,2hop__690412_526810,"[{'idx': 4, 'title': 'Pa Sak Jolasid Dam', 'pa...",For what river does the river on which Pa Sak ...,"[{'id': 690412, 'question': 'Pa Sak Jolasid Da...",Chao Phraya River,[Chao Phraya River],True,[Chao Phraya River]
3,2hop__263638_69048,"[{'idx': 5, 'title': 'Rosaline Bozimo', 'parag...",Who is the Chief Judge of the Tebesa Nemine's ...,"[{'id': 263638, 'question': 'Tebesa Nemine >> ...",Honorable Justice Abiodun Smith,[Honorable Justice Abiodun Smith],True,[Honorable Justice Abiodun Smith]
4,2hop__142842_68489,"[{'idx': 2, 'title': 'Snapper Foster', 'paragr...",Who did the performer of Night Rocker play on ...,"[{'id': 142842, 'question': 'Which performer r...",Snapper Foster,[Snapper Foster],True,[Snapper Foster]


In [19]:
for i in range(1,N_RUNS+1):
    df_llama_sft, scores = benchmark(df_llama_sft, qa_func, perfect_retrieval_func, ignore_errors=True)
    results.append({**scores, "retrieval": "groundtruth", "context": "triplets", "jerx": "llama-sft", "run": i})
    jprint(scores)

  0%|          | 0/100 [00:00<?, ?it/s]

{
  "exact_match": 0.56,
  "f1": 0.6488034188034187,
  "fuzzy_match": 0.67
}


## gpt-4-turbo

In [20]:
# from bellek.jerx.fewshot.llm import make_kg_triplet_extract_fn

# extract_kg_triplets = make_kg_triplet_extract_fn(model='gpt-4-turbo')

# def replace_paragraphs(row):
#     new_paragraphs = []
#     for p in row['paragraphs']:
#         p = deepcopy(p) 
#         triplets_str = '\n'.join(" | ".join(triplet) for triplet in extract_kg_triplets(p['paragraph_text']))
#         p['paragraph_text'] = '\n'.join(["# Entity-relation-entity triplets", triplets_str])
#         new_paragraphs.append(p)
#     row['paragraphs'] = new_paragraphs
#     return row

# df_gpt = df.progress_apply(replace_paragraphs, axis=1) 
# df_gpt.head()
# print(df_gpt.iloc[0]['paragraphs'][2]['paragraph_text'])

In [21]:
# _, scores = benchmark(df_gpt, qa_func, perfect_retrieval_func, ignore_errors=True)
# results.append({**scores, "retrieval": "groundtruth", "context": "triplets", "jerx": "gpt-4-turbo"})
# jprint(scores)

# Report

In [22]:
pd.options.display.float_format = '{:,.3f}'.format

In [23]:
report_df = pd.DataFrame.from_records(results, columns=['run', 'context', 'retrieval', 'jerx', 'exact_match', 'fuzzy_match', 'f1'])
report_df.drop(columns=['context', 'retrieval', 'fuzzy_match'])

,run,jerx,exact_match,f1
0,1,llama-zero-shot,0.560,0.629
1,1,llama-few-shot,0.540,0.656
2,1,llama-sft,0.560,0.649


In [24]:
report_df[['jerx', 'exact_match', 'f1']].groupby('jerx').mean().loc[['llama-zero-shot', 'llama-few-shot', 'llama-sft']]

,exact_match,f1
jerx,,
llama-zero-shot,0.560,0.629
llama-few-shot,0.540,0.656
llama-sft,0.560,0.649
